In [5]:
from lib.dataset.mesh_util import *
#from lib.smplx.body_models import SMPLX
from lib.dataset.TestDataset import TestDataset
from lib.common.config import cfg
from lib.pymafx.core import path_config
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
SMPLX_object = SMPLX()

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

Type of each parameter in 'dataset_param':

    <class 'str'>
    <class 'NoneType'>
    <class 'bool'>
    <class 'str'>
    <class 'int'>
    <class 'bool'>

In [8]:
dataset_param = {
        "image_dir": './examples/testimg/',
        "seg_dir": None,
        "use_seg": True,    # w/ or w/o segmentation
        "hps_type": 'pymafx' ,   # pymafx/pixie
        "vol_res": cfg.vol_res,
        "single": False, #args.multi,
    }


In [9]:
path_config.pymafx_data_dir = "./data/HPS/pymafx_data"

os.path.exists(path_config.pymafx_data_dir)

True

In [10]:
param_npz = np.load('./00006_0.npz' , allow_pickle=True)
print(type(param_npz))
for item in param_npz:
    print (item)

<class 'numpy.lib.npyio.NpzFile'>
global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [11]:
param = {key: torch.as_tensor(param_npz[key]) for key in param_npz.keys()}

In [12]:
for item in param_npz:
    print (item)

global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [13]:

for key in param.keys():
    print(key)

global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [14]:
from apps.infer import view_dict_items

In [15]:
view_dict_items(param)

global_orient: torch.Size([1, 3])
body_pose: torch.Size([21, 3])
left_hand_pose: torch.Size([15, 3])
right_hand_pose: torch.Size([15, 3])
jaw_pose: torch.Size([1, 3])
leye_pose: torch.Size([1, 3])
reye_pose: torch.Size([1, 3])
betas: torch.Size([1, 10])
expression: torch.Size([1, 10])
transl: torch.Size([1, 3])


In [16]:
#SMPLX_object = SMPLX('00006_0.npz')

In [17]:
data = np.load('./results/econ/obj/000023.jpg_0.npz',allow_pickle=True)
#print(data)
#data= data.tolist()
for item in data:
    print(item,': ',data[item].shape)

#print(data['scale'])

global_orient :  (1, 3)
body_pose :  (21, 3)
left_hand_pose :  (15, 3)
right_hand_pose :  (15, 3)
jaw_pose :  (1, 3)
leye_pose :  (1, 3)
reye_pose :  (1, 3)
betas :  (1, 10)
expression :  (1, 10)
transl :  (1, 3)


In [18]:
npyfile = './pymafx_results/econ/obj/OIP_smpl_00.npy'
NpzFile = './00006_0.npz'

In [19]:
data = np.load(NpzFile, allow_pickle=True)

In [20]:
for item in data :
    print(item)

global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [28]:
from axis2matrix import axis_angle_to_rotation_matrix
from lib.kornia.core import Tensor

In [29]:
matrix_jaw = axis_angle_to_rotation_matrix( Tensor(data['jaw_pose']))

In [30]:
print(matrix_jaw)

tensor([[[ 1.0000e+00,  5.4224e-04, -2.6588e-03],
         [-5.8650e-04,  9.9986e-01, -1.6671e-02],
         [ 2.6494e-03,  1.6673e-02,  9.9986e-01]]])


In [21]:
smpl_mesh,smpl_joint = load_fit_body(fitted_path=NpzFile,scale=1)

<class 'numpy.ndarray'>
torch.Size([15, 3])
global_orient:torch.Size([1, 3])
body_pose:torch.Size([21, 3])
jaw_pose:torch.Size([1, 3])
leye_pose:torch.Size([1, 3])
reye_pose:torch.Size([1, 3])
left_hand_pose:torch.Size([15, 3])
right_hand_pose:torch.Size([15, 3])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 21 for tensor number 1 in the list.